# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846719529136                   -0.70    5.0         
  2   -7.852305297516       -2.25       -1.53    1.0   22.9ms
  3   -7.852606962965       -3.52       -2.57    1.2   24.1ms
  4   -7.852645918514       -4.41       -2.85    2.8   32.8ms
  5   -7.852646413359       -6.31       -3.06    1.0   23.4ms
  6   -7.852646675903       -6.58       -3.95    1.0   23.8ms
  7   -7.852646686058       -7.99       -4.88    1.5   25.8ms
  8   -7.852646686714       -9.18       -5.29    2.2   30.3ms
  9   -7.852646686728      -10.83       -5.86    1.2   24.4ms
 10   -7.852646686730      -11.96       -6.00    1.5   87.4ms
 11   -7.852646686730      -12.50       -6.64    1.0   23.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846708431358                   -0.70           4.5         
  2   -7.852522239731       -2.24       -1.64   0.80    2.0    241ms
  3   -7.852633976178       -3.95       -2.75   0.80    1.0   22.0ms
  4   -7.852646570916       -4.90       -3.19   0.80    2.2   28.1ms
  5   -7.852646680461       -6.96       -4.01   0.80    1.0   21.5ms
  6   -7.852646686269       -8.24       -4.80   0.80    1.2   22.1ms
  7   -7.852646686713       -9.35       -5.80   0.80    2.0   26.8ms
  8   -7.852646686730      -10.78       -6.45   0.80    2.2    103ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.416209e+01     3.774023e+00
 * time: 0.41423797607421875
     1     1.572782e+00     1.776811e+00
 * time: 0.6043200492858887
     2    -1.538706e+00     2.105023e+00
 * time: 0.627140998840332
     3    -3.765856e+00     1.730844e+00
 * time: 0.6597449779510498
     4    -5.138614e+00     1.608031e+00
 * time: 0.6923348903656006
     5    -6.825487e+00     9.754160e-01
 * time: 0.7247829437255859
     6    -7.448776e+00     3.885392e-01
 * time: 0.7574069499969482
     7    -7.677616e+00     2.056821e-01
 * time: 0.7798240184783936
     8    -7.758696e+00     1.007179e-01
 * time: 0.802253007888794
     9    -7.796301e+00     1.181002e-01
 * time: 0.8246760368347168
    10    -7.821420e+00     7.343428e-02
 * time: 0.8474099636077881
    11    -7.836631e+00     5.281139e-02
 * time: 0.8700239658355713
    12    -7.843567e+00     4.812373e-02
 * time: 0.8923778533935547
    13    -7.847283e+00     3.674673e-02
 * time: 0.91479206085

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846773398731                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645856426                   -1.64         
  2   -7.852646686730       -6.08       -3.70    1.55s
  3   -7.852646686730      -13.25       -7.21    195ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.366495997954785e-7
|ρ_newton - ρ_scfv| = 2.138509186866559e-7
|ρ_newton - ρ_dm|   = 1.7795113784938147e-9
